<a href="https://colab.research.google.com/github/mrchocha/CSE523-Machine-Learning-Fantastic-4/blob/main/Fantastic_4_DataCollectionUsingSNScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data collection for by days for various DIJA indices.

##Importing SnScrape and other libraries.

In [ ]:
pip install snscrape

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from textblob import TextBlob
import re
from google.colab import files
import os, shutil

## Functions for NLP

In [ ]:
# Create a function to compute negative (-1), neutral (0) and positive (+1) analysis
def getAnalysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'

def cleanText(text):
 text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
 text = re.sub('#', '', text) # Removing '#' hash tag
 text = re.sub('RT[\s]+', '', text) # Removing RT
 text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
 
 return text

# Create a function to get the subjectivity
def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
   return  TextBlob(text).sentiment.polarity

#Fully Automated Code for Data collection

##Funciton for automating this data collection.

In [ ]:
year = 2010
#----------------------------change here----------------------------
month = 6

def createFromDate(day):
  ans = ""+str(year)+"-"
  ans += str(month).zfill(2)+"-"
  ans += str(day).zfill(2)
  return ans

def createEndDate(day):
  ans = ""+str(year)+"-"
  ans += str(month).zfill(2)+"-"
  ans += str(day).zfill(2)
  return ans

def monthGen():
  months = ['Jan', 'Feb', 'March', 'April', 'May', 'June', 'July', 'August', 'Sep', 'Oct', 'Nov', 'Dec']
  return months[month-1]

def createFileName(company, day):
  ans = company+"_"
  ans += str(day).zfill(2)+"_"
  ans += monthGen()+"_"
  ans += str(year)
  ans += ".csv"
  return ans

def createQuery(company, day):
  ans = "$"
  ans += company
  ans += " since:"
  ans += createFromDate(day)
  ans += " until:"
  ans += createEndDate(day+1)
  return ans

## Firing Query for only one Day.

In [ ]:
maxTweets = 10000
tweets = []
companies = ["AMZN", "AAPL", "T", "DELL", "EBAY", "GOOG", "IBM", "INTC", "MSFT", "ORCL", "SSNLF", "SAP", "YHOO"]

for com in companies:
  #----------------------------change here----------------------------
  q = "$"+com+" since:2010-06-30 until:2010-07-01"
  for i,tweet in enumerate(sntwitter.TwitterSearchScraper(q).get_items()):
      if i>maxTweets:
          break
      tweets.append([tweet.url, tweet.date, tweet.id, tweet.content])

  # Making a dataframe of the data we got from the SNScrape.
  df_tweets = pd.DataFrame(tweets, columns=['Url', 'Datetime', 'Tweet_Id', 'Text'])

  # Cleaning the 'text' field from the dataframe.
  # Creating 2 columns of Subjectivity and Polarity.
  # Creating a column having Labels Pos, Neg, Nut.
  df_tweets['Text'] = df_tweets['Text'].apply(cleanText)
  df_tweets['Subjectivity'] = df_tweets['Text'].apply(getSubjectivity)
  df_tweets['Polarity'] = df_tweets['Text'].apply(getPolarity)
  df_tweets['Label'] = df_tweets['Polarity'].apply(getAnalysis)

  #----------------------------change here----------------------------
  df_tweets.to_csv(createFileName(com, 30))
  # files.download(fname)
  print(createFileName(com, 31))

##Automating the dowanload of data of all days.

In [ ]:
maxTweets = 10000
tweets = []
cnt = 0

companies = ["AMZN", "AAPL", "T", "DELL", "EBAY", "GOOG", "IBM", "INTC", "MSFT", "ORCL", "SSNLF", "SAP", "YHOO"]

for com in companies:

  #----------------------------change here----------------------------
  for day in range(1,30):

    tweets = []
    q = createQuery(com, day)
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(q).get_items()):
      if i>maxTweets:
          break
      tweets.append([tweet.url, tweet.date, tweet.id, tweet.content])
    
    df_tweets = pd.DataFrame(tweets, columns=['Url', 'Datetime', 'Tweet_Id', 'Text'])

    df_tweets['Text'] = df_tweets['Text'].apply(cleanText)
    df_tweets['Subjectivity'] = df_tweets['Text'].apply(getSubjectivity)
    df_tweets['Polarity'] = df_tweets['Text'].apply(getPolarity)
    df_tweets['Label'] = df_tweets['Polarity'].apply(getAnalysis)

    df_tweets.to_csv(createFileName(com, day))
    print(cnt, ". ",createFileName(com, day), " successfully created.")
    cnt += 1
    # files.download(createFileName(com, day))

In [ ]:
path = './'+monthGen()+"_"+str(year)
os.mkdir(path)

In [ ]:
src_fname = "./"+monthGen()+"_"+str(year)
down_fname = src_fname+".zip"

print(down_fname)
print(src_fname)

./June_2010.zip
./June_2010


In [ ]:
companies = ["AMZN", "AAPL", "T", "DELL", "EBAY", "GOOG", "IBM", "INTC", "MSFT", "ORCL", "SSNLF", "SAP", "YHOO"]

for com in companies:
  #----------------------------change here----------------------------
  for day in range(1,31):
    shutil.move(createFileName(com, day), src_fname)
    print(createFileName(com, day), " moved successfully.")

In [ ]:
shutil.make_archive(src_fname, 'zip', src_fname)

'/content/June_2010.zip'

In [ ]:
files.download(down_fname)